---
title: How to submit a PwBaseWorkChain 
subtitle: 
#author:
#  - name: Miki Bonacci
    #affiliations: Executable Books; Curvenote
    #orcid: 0000-0002-7859-8394
#    email: miki.bonacci@psi.ch
license:
  code: MIT
#date: 2023/01/23
---

In the following we show how to run a calculation with the `aiida-quantumespresso` plugin using the new *atomistic* `StructureData`. 
The procedure is very similar to the [old way](https://aiida-quantumespresso.readthedocs.io/en/latest/tutorials/first_pw.html), except to the fact that now properties like magnetization and charge are provided within the structure
and not in the input parameters of the `PwCalculation`.

***For now, only magnetization is supported.***

***PLEASE USE THE FOLLOWING VERSIONS OF THE RELATED CODES:***

- aiida-atomistic: [develop branch](https://github.com/mikibonacci/aiida-atomistic/tree/develop)
- aiida-core: [new_StructureData branch](https://github.com/mikibonacci/aiida-core/tree/new_StructureData)
- aiida-quantumespresso: [new_StructureData branch](https://github.com/mikibonacci/aiida-quantumespresso/tree/new_StructureData)
- aiida-pseudo: [new_StructureData branch](https://github.com/mikibonacci/aiida-pseudo/tree/new_StructureData)

In [1]:
from aiida import load_profile, orm
from aiida_quantumespresso.workflows.pw.base import PwBaseWorkChain
from aiida_atomistic import StructureData, StructureDataMutable

load_profile()

Profile<uuid='1a5a8d0836814a04a238c67cc7481655' name='default'>

Once loaded the necessary modules, we start building up our `StructureDataMutable`, to be then converted into the AiiDA `StructureData` (via the `to_immutable` method):

In [2]:
mutable_structure = StructureDataMutable()
mutable_structure.set_cell([[0.0, 1.8, 1.8], [1.8, 0.0, 1.8], [1.8, 1.8, 0.0]])

mutable_structure.add_atom({
            'symbol':'Si',
            'position':[1/2, 1/2, 1/2],
            'kind_name': 'Si1'
        })
mutable_structure.add_atom({
            'symbol':'Si',
            'position':[3/4, 3/4, 3/4],
            'kind_name': 'Si1'
        })

structure = mutable_structure.to_immutable()

We then generate the `PwBaseWorkChain` builder via the `get_builder_from_protocol` method, providing as input the *pw.x* code node and the structure:

In [3]:
from aiida.engine import submit, run_get_node


builder = PwBaseWorkChain.get_builder_from_protocol(
    code=orm.load_code("pw-qe-7.2@localhost"), 
    structure=structure,
    protocol="fast",
    )

run = run_get_node(builder)

09/09/2024 01:41:24 PM <2191820> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1539|PwBaseWorkChain|run_process]: launching PwCalculation<1544> iteration #1
09/09/2024 01:41:28 PM <2191820> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1539|PwBaseWorkChain|results]: work chain completed after 1 iterations
09/09/2024 01:41:29 PM <2191820> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1539|PwBaseWorkChain|on_terminated]: remote folders will not be cleaned


After the job is finished, you can inspect the outputs as usual:

In [4]:
run[0]

{'remote_folder': <RemoteData: uuid: 6593814c-d048-4f3b-9f74-3f311f5e722b (pk: 1545)>,
 'retrieved': <FolderData: uuid: 9a9b7cf5-e4b2-4735-8f8e-acd46b7e8743 (pk: 1546)>,
 'output_parameters': <Dict: uuid: 85c053e8-f24d-42f6-919f-8081b551ca82 (pk: 1549)>,
 'output_trajectory': <TrajectoryData: uuid: 5893d1f2-79fb-4f65-8614-178d434eb2e3 (pk: 1548)>,
 'output_band': <BandsData: uuid: 285c376d-78ba-49fd-b6b2-78164cf26498 (pk: 1547)>}

## How to deal with magnetic configurations

The first way to prepare a magnetic calculation is to trigger the `nspin` or `noncolin` in the input parameters of the `PwCalculation` respectively for *collinear* (`nspin=2`) and *non collinear* (`noncolin=.True.`) calculations.

Of course, the magnetic moments have to be provided within the `StructureData`, so that they can be used to build the [*starting_magnetization*](https://www.quantum-espresso.org/Doc/INPUT_PW.html#idm301) variable in the `&SYSTEM` namelist.


### Collinear case 

We first load the structure using Pymatgen to parse the *mcif* file for BCC Iron:

In [5]:
from pymatgen.core import Structure

#suppose you are in the `aiida-atomistic` root folder
iron_bcc = Structure.from_file('../examples/structure/data/Fe_bcc.mcif', primitive=True)
magnetic_structure = StructureData.from_pymatgen(iron_bcc)

print(magnetic_structure.properties.magmoms)

[[0.0, 0.0, 2.5]]


In [6]:
magnetic_structure.properties.kinds

['Fe0']

We can check that our structure is collinear (trivial in this case):

In [7]:
magnetic_structure.is_collinear

True

For now, we consider only collinear magnetic moments along the z axis.

We can create again the builder as before, this time overriding `nspin` to be equal to 2 (*collinear* calculation):

In [8]:
builder = PwBaseWorkChain.get_builder_from_protocol(
    code=orm.load_code("pw-qe-7.2@localhost"), 
    structure=magnetic_structure,
    protocol="fast",
    overrides={
        "pw":{
            "parameters":{
                "SYSTEM":
                    {"nspin": 2}
            }
        }
    }
    )

In [9]:
run = run_get_node(builder)

09/09/2024 01:41:31 PM <2191820> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1556|PwBaseWorkChain|run_process]: launching PwCalculation<1561> iteration #1
09/09/2024 01:41:39 PM <2191820> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1556|PwBaseWorkChain|results]: work chain completed after 1 iterations
09/09/2024 01:41:40 PM <2191820> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1556|PwBaseWorkChain|on_terminated]: remote folders will not be cleaned


### The output magnetic moments

For now, the new output magnetization of a structure will not be stored in the output `StructureData` (if any). 
The reason is that the output magnetization can present some discrepancies which are actually artifacts of the simulation. For example, symmetries maybe not correctly detected (due to different kinds) and so magnetization magnitudes (and orientations) are not the same as expected. 

So, the if the user needs it, he can use a `calcfuntion` to generate a new `StructureData` with the new magnetic moments (both as list of *floats* or of len=3 *lists*):

In [10]:
trajectory = run[-1].called[-1].outputs.output_trajectory # run[1].called[-1].outputs.output_trajectory
magnetic_moments = [magmom[0] for magmom in trajectory.get_array('atomic_magnetic_moments')]
print(magnetic_moments)

[2.3254]


In [11]:
from aiida_quantumespresso.utils.magnetic import generate_structure_with_magmoms

new_structure = generate_structure_with_magmoms(magnetic_structure, magnetic_moments)
print(f"New StructureData: {new_structure}\nAttached magmoms: {new_structure.properties.magmoms}")

New StructureData: uuid: 89ad3b8e-0d2c-4370-ba83-43a4a2b801cb (pk: 1569)
Attached magmoms: [[0.0, 0.0, 2.3254]]


Using this utility function, kinds will be automatically detected. 

If you don't want the provenance and the new `StructureData` node to be stored, add `metadata={"store_provenance": False}` to the input of the `calcfunction`. 

### Non-collinear case 

For the non-collinear case, we just need to put `noncolin: True` in the input parameters:

In [12]:
builder = PwBaseWorkChain.get_builder_from_protocol(
    code=orm.load_code("pw-qe-7.2@localhost"), 
    structure=magnetic_structure,
    protocol="fast",
    overrides={
        "pw":{
            "parameters":{
                "SYSTEM":
                    {"noncolin": True}
            }
        }
    }
    )

In [13]:
run = run_get_node(builder)

09/09/2024 01:41:41 PM <2191820> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1575|PwBaseWorkChain|run_process]: launching PwCalculation<1580> iteration #1
09/09/2024 01:41:59 PM <2191820> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1575|PwBaseWorkChain|results]: work chain completed after 1 iterations
09/09/2024 01:42:00 PM <2191820> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1575|PwBaseWorkChain|on_terminated]: remote folders will not be cleaned


In [14]:
run[0]

{'remote_folder': <RemoteData: uuid: b18959ed-1830-487c-9f47-6a104556a7bd (pk: 1581)>,
 'retrieved': <FolderData: uuid: 181b89d9-3864-411a-9fed-51bd87f081e5 (pk: 1582)>,
 'output_parameters': <Dict: uuid: e366238f-ed04-4e8c-9eea-cad64f7daf55 (pk: 1585)>,
 'output_trajectory': <TrajectoryData: uuid: 78d024fd-a28f-43d2-ac8e-de4f691f882e (pk: 1584)>,
 'output_band': <BandsData: uuid: 688b8c70-a688-46bd-8f9a-ea73810c0962 (pk: 1583)>}

Currently, no parser for non-collinear magnetization is implemented in the `aiida-quantumespresso` plugin.